In [50]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
from mediapipe.python.solutions.drawing_utils import draw_landmarks
from mediapipe.python.solutions import drawing_styles
import numpy as np

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from util import mediapipe_utils
from util import opencv_utils

import importlib



importlib.reload(mediapipe_utils)
importlib.reload(opencv_utils)

from util.mediapipe_utils import *
from util.opencv_utils import *

%load_ext autoreload
%autoreload 2

# Extract and display first frame
# Daniel Path
video_path = r"C:\Users\User\OneDrive\Documents\Projects\Lip-Reading\GRID\s1\s1\bbaf2n.mpg"
# Ryan Path
# video_path = r"C:\Projects\Lip_Reading\GRID\s1\bbaf2n.mpg"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# Isolate lip landmarks and coordinates
# Outer lips (full loop)
outer_lips = [
    61, 146, 91, 181, 84, 17,
    314, 405, 321, 375, 291, 409, 
    270, 269, 267, 0, 37, 39, 40, 185
]

# Inner lips (inside mouth opening)
inner_lips = [
    78, 95, 88, 178, 87, 
    14, 317, 402, 318, 324, 
    308, 415, 310, 311, 312, 
    13, 82, 81, 80, 191
]

inner_mouth = [
    184, 183, 74, 42, 73, 41, 72, 38,
    11, 12, 302, 268, 303, 271, 304, 272,408,
    407, 96, 77, 89, 90, 179, 180, 86, 85, 15, 
    16, 316, 315, 403, 404, 319, 320, 325, 307
]

lower_jaw = [
    148, 152, 377
]

In [52]:
def setup_landmarks(video_path, frame_number):
    # Crop and proccess landmarks
    frame = load_video(video_path)[frame_number]
    landmarks = get_landmarks(frame)[0] # 0 being the first face found
    coords = convert_landmarks_to_coordinates(landmarks, frame.shape)
    x_min, y_min, x_max, y_max = get_bounding_box(coords, frame.shape)
    cropped_frame = frame[int(y_min):int(y_max), int(x_min):int(x_max)]
    
    cropped_landmarks = get_landmarks(cropped_frame)[0]
    h, w, _ = cropped_frame.shape
    coords = convert_landmarks_to_coordinates(cropped_landmarks, (h, w))
    
    return (cropped_landmarks, coords, h, w)

In [53]:
# Normalized Features
# Divide by dimensions of cropped face to account for face sizes 
# Use face height and eye distance as normailization metrics
    
def normalize_vertical_measurements(coords, measurements):
    face_height = landmark_dist(coords[10], coords[152])
    return [m/face_height for m in measurements]

def normalize_horizontal_measurements(coords, measurements):
    eye_dist = landmark_dist(coords[133], coords[362])
    return [m/eye_dist for m in measurements]

def normalize_area(coords, measurements):
    face_height = landmark_dist(coords[10], coords[152])
    eye_dist = landmark_dist(coords[133], coords[362])
    return [m/(face_height*eye_dist) for m in measurements]
    

In [54]:
def get_primary_measurements(video_path, frame_number):

    landmarks, coords, h, w = setup_landmarks(video_path, frame_number)

    upper_vermillion_height = landmark_dist(coords[13], coords[0])
    lower_verillion_height = landmark_dist(coords[14], coords[17])

    vert_distance = landmark_dist(coords[13], coords[14])
    hori_distance = landmark_dist(coords[308], coords[78])
    aspect_ratio = hori_distance/vert_distance

    lower_jaw_distance = landmark_dist(coords[14], coords[152])

    inner_mouth_area = abs(area_of_points(inner_lips, coords))
    outer_mouth_area = abs(area_of_points(outer_lips, coords))

    norm_uvh, norm_lvh, norm_vd, norm_jd = normalize_vertical_measurements(coords, [upper_vermillion_height,lower_verillion_height,vert_distance,lower_jaw_distance])
    norm_hd = normalize_horizontal_measurements(coords, [hori_distance])
    norm_ar = norm_hd/norm_vd
    norm_ima, norm_oma = normalize_area(coords, [inner_mouth_area, outer_mouth_area])

    return [
        upper_vermillion_height,
        lower_verillion_height,
        vert_distance,
        hori_distance,
        aspect_ratio,
        lower_jaw_distance,
        inner_mouth_area,
        outer_mouth_area,
        norm_uvh,
        norm_lvh,
        norm_vd,
        norm_hd,
        norm_ar,
        norm_jd,
        norm_ima,
        norm_oma
    ]

In [55]:
# Get Avgeraged Measurements
# Average over 3 frames to get a more stable measurement

# [upper_vermillion_height, lower_vermillion_height, vert_distance, hori_distance, aspect_ratio, lower_jaw_distance, inner_mouth_area, outer_mouth_area]
# [norm_uvh, norm_lvh, norm_vd, norm_hd, norm_ar, norm_jd, norm_ima, norm_oma]
def get_average_resting(video_path):
    avg_features = []
    for i in range(3): # Average of first three frames
        vals = get_primary_measurements(video_path,i)
        for j in range(16):
            avg_features += vals[j]
    avg_features = [x / 3 for x in avg_features]
    return avg_features



In [56]:
def get_frame_measurement(video_path, frame_number, data_table , average_resting_vals):    
    # Primary measurements
    upper_vermillion_height,lower_verillion_height,vert_distance,hori_distance,aspect_ratio,lower_jaw_distance,inner_mouth_area,outer_mouth_area,norm_uvh,norm_lvh,norm_vd,norm_hd,norm_ar,norm_jd,norm_ima,norm_oma = get_primary_measurements(video_path, frame_number)

    landmarks, coords, h, w = setup_landmarks(video_path, frame_number)
    
    # Temporal landmarks and scientific - engineered landmarks

        # Vertical Lip Compression
        # - Measures how "pursed" the lips are
        
        # Corner Displacement
        # - Measures how tightened mouth is
        
        # Key Landmark Velocities 
        # - Track velocity of key landmarks using x2-x1 / t2-t1, such as lip corners, top and bottom of inner edge of lip, etc.

    if(frame_number == 0): # grab average resting values from begining of video
        average_resting_vals = get_average_resting(video_path)

    vert_lip_compression = (upper_vermillion_height + lower_verillion_height) / (average_resting_vals[0] + average_resting_vals[1])
    corner_displacement = hori_distance / average_resting_vals[3]
    
    norm_vlc = normalize_vertical_measurements(coords, [vert_lip_compression])
    norm_cd = normalize_horizontal_measurements(coords, [corner_displacement])
    
    if frame_number == 0:
        lip_corner_velocity = 0
        top_inner_lip_velocity = 0
        bottom_inner_lip_velocity = 0
    else:
        lip_corner_velocity = hori_distance - data_table[frame_number - 1][3] 
        top_inner_lip_velocity = upper_vermillion_height - data_table[frame_number - 1][0]
        bottom_inner_lip_velocity = lower_verillion_height - data_table[frame_number - 1][1]
    
    return([
        frame_number,
        float(upper_vermillion_height),
        float(lower_verillion_height),
        float(vert_distance),
        float(hori_distance),
        float(aspect_ratio),
        float(lower_jaw_distance),
        float(inner_mouth_area),
        float(outer_mouth_area),
        float(norm_uvh),
        float(norm_lvh),
        float(norm_vd),
        float(norm_hd),
        float(norm_ar),
        float(norm_jd),
        float(norm_ima),
        float(norm_oma),
        float(norm_vlc),
        float(norm_cd),
        float(vert_lip_compression),
        float(norm_vlc),
        float(corner_displacement),
        float(norm_cd),
        float(lip_corner_velocity),
        float(top_inner_lip_velocity),
        float(bottom_inner_lip_velocity)
    ])


In [57]:
# Make 2D array of measurements
# Each row is a frame, each column is a measurement
video_path = r"C:\Users\User\OneDrive\Documents\Projects\Lip-Reading\GRID\s1\s1\bbaf2n.mpg"
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
data_table = [[]]
average_resting_vals = get_average_resting(video_path)
for i in range(total_frames):
    frame_data = get_frame_measurement(video_path,i,data_table, average_resting_vals)
    print(f"Frame {i}: {frame_data}")
    print(f"measurements_2D_arr: {measurements_2D_arr}")
    measurements_2D_arr[i].append(frame_data)
    



IndexError: list index out of range